In [1]:
SEED = 42
import numpy as np
import pandas as pd
from utils import read_train_test, get_time_stock
from optiver_lofo import plot_importance, OptiverRecursiveLOFO
import itertools
import gc

/home/yuchen/kaggle/kaggle-optiver/working/optiver_lofo.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
train, _ = read_train_test()
df_book = pd.read_csv('../input/processed-book-ffill/df_book.csv')
df_trade = pd.read_csv('../input/processed-book-ffill/df_trade.csv')
train_ = df_book.merge(df_trade, on = ['row_id'], how = 'left')
train = train.merge(train_, on = ['row_id'], how = 'left')
del _, df_book, df_trade, train_
gc.collect()
# Get group stats of time_id and stock_id
train = get_time_stock(train)
train = train.sample(frac=1, random_state=SEED).reset_index(drop=True)
train

Our training set has 428932 rows


,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap1_median,wap1_kurtosis,wap1_skew,...,trade_log_return_realized_volatility_300_mean_time,trade_log_return_realized_volatility_300_std_time,trade_log_return_realized_volatility_300_max_time,trade_log_return_realized_volatility_300_min_time,trade_log_return_realized_volatility_300_median_time,trade_log_return_realized_volatility_150_mean_time,trade_log_return_realized_volatility_150_std_time,trade_log_return_realized_volatility_150_max_time,trade_log_return_realized_volatility_150_min_time,trade_log_return_realized_volatility_150_median_time
0,16,9787,0.005983,16-9787,600.47920,1.000799,0.001474,1.001224,-0.717402,-0.603572,...,0.002483,0.000756,0.004954,0.001089,0.002451,0.003181,0.000859,0.006486,0.001580,0.003066
1,105,5773,0.002415,105-5773,600.64197,1.001070,0.000766,1.000880,-1.233713,0.323047,...,0.001704,0.000593,0.004029,0.000843,0.001570,0.002196,0.000715,0.005209,0.001141,0.002004
2,58,22076,0.002863,58-22076,598.30756,0.997179,0.000907,0.996946,-0.797297,0.444533,...,0.000838,0.000478,0.003798,0.000237,0.000727,0.001039,0.000544,0.004101,0.000335,0.000903
3,77,5817,0.005279,77-5817,601.05817,1.001764,0.002921,1.001746,-0.907996,-0.267860,...,0.004393,0.001471,0.009818,0.001796,0.004206,0.005367,0.001700,0.011079,0.002390,0.005120
4,56,14572,0.002328,56-14572,598.78840,0.997981,0.000927,0.997882,-0.857376,0.054103,...,0.000915,0.000401,0.002313,0.000221,0.000834,0.001135,0.000482,0.002866,0.000348,0.001030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,76,21439,0.002647,76-21439,600.85600,1.001427,0.000982,1.001238,-0.258683,0.731784,...,0.001346,0.000629,0.003775,0.000000,0.001217,0.001646,0.000698,0.004178,0.000520,0.001442
428928,108,16733,0.001089,108-16733,599.87500,0.999792,0.000372,0.999724,-1.149882,-0.009472,...,0.000877,0.000352,0.002123,0.000000,0.000857,0.001113,0.000443,0.002451,0.000000,0.001026
428929,37,14273,0.002531,37-14273,601.11096,1.001852,0.000367,1.001833,0.624756,1.055845,...,0.000989,0.000426,0.002689,0.000000,0.000896,0.001216,0.000498,0.002931,0.000000,0.001113
428930,41,11351,0.001864,41-11351,600.30000,1.000500,0.000194,1.000479,-0.102242,-0.335757,...,0.001173,0.000534,0.003743,0.000489,0.001038,0.001426,0.000638,0.004545,0.000618,0.001287


In [3]:
train_ = pd.read_csv("../baseline/data/baseline_train.csv")
train = train.merge(train_, on=["time_id", "stock_id", "target"], how="inner")
train['stock_id'] = train['stock_id'].astype('category')
train

,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap1_median,wap1_kurtosis,wap1_skew,...,__book_wap_mean_lambda_____mean___stock,__book_wap_mean_lambda_____std___stock,__book_wap_mean_lambda_____max___stock,__book_wap_mean_lambda_____min___stock,__book_wap_mean_lambda_____median___stock,__book_wap_diff_lambda_____mean___stock,__book_wap_diff_lambda_____std___stock,__book_wap_diff_lambda_____max___stock,__book_wap_diff_lambda_____min___stock,__book_wap_diff_lambda_____median___stock
0,16,9787,0.005983,16-9787,600.47920,1.000799,0.001474,1.001224,-0.717402,-0.603572,...,0.004312,0.003476,0.052871,0.000639,0.003271,NaN,NaN,NaN,1.764698,9.811453
1,105,5773,0.002415,105-5773,600.64197,1.001070,0.000766,1.000880,-1.233713,0.323047,...,0.003004,0.002463,0.037666,0.000510,0.002254,NaN,NaN,NaN,0.000000,13.445859
2,58,22076,0.002863,58-22076,598.30756,0.997179,0.000907,0.996946,-0.797297,0.444533,...,0.004020,0.003281,0.040993,0.000358,0.003100,NaN,NaN,NaN,2.099306,12.351617
3,77,5817,0.005279,77-5817,601.05817,1.001764,0.002921,1.001746,-0.907996,-0.267860,...,0.003872,0.002671,0.036644,0.000849,0.003146,NaN,NaN,NaN,0.921629,13.606591
4,56,14572,0.002328,56-14572,598.78840,0.997981,0.000927,0.997882,-0.857376,0.054103,...,0.004412,0.002564,0.026125,0.001012,0.003628,NaN,NaN,NaN,5.596212,23.238896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,76,21439,0.002647,76-21439,600.85600,1.001427,0.000982,1.001238,-0.258683,0.731784,...,0.004077,0.002744,0.037967,0.000780,0.003275,NaN,NaN,NaN,5.147635,22.639032
428928,108,16733,0.001089,108-16733,599.87500,0.999792,0.000372,0.999724,-1.149882,-0.009472,...,0.003629,0.002615,0.034763,0.000405,0.002935,NaN,NaN,NaN,0.000000,12.628781
428929,37,14273,0.002531,37-14273,601.11096,1.001852,0.000367,1.001833,0.624756,1.055845,...,0.006088,0.004394,0.055484,0.000688,0.004739,NaN,NaN,NaN,0.655598,9.141956
428930,41,11351,0.001864,41-11351,600.30000,1.000500,0.000194,1.000479,-0.102242,-0.335757,...,0.002461,0.001852,0.020295,0.000328,0.001882,NaN,NaN,NaN,0.000000,12.104390


In [52]:
feature_cols = [c for c in train.columns if c not in ['row_id', 'target', 'time_id', 'target', 'logtarget']]
# wap1_cols = [c for c in feature_cols if c.split('_')[0]=='wap1']
# feature_cols = [c for c in feature_cols if c not in wap1_cols]
print(f"# features: {len(feature_cols)}")
feature_groups = {
    'wap1': [c for c in feature_cols if c.split('_')[0]=='wap1'], # removed
    
    'wap2': [c for c in feature_cols if c.split('_')[0]=='wap2'],
    'log_return1': [c for c in feature_cols if c.split('_')[0:2]==['log', 'return1'] and c.split('_')[-1] not in ['time', 'stock']],
    'log_return2': [c for c in feature_cols if c.split('_')[0:2]==['log', 'return2'] and c.split('_')[-1] not in ['time', 'stock']],
    'wap_balance': [c for c in feature_cols if c.split('_')[0:2]==['wap', 'balance']],
    'price_spread': [c for c in feature_cols if c.split('_')[0:2]==['price', 'spread']],
    'bid_spread': [c for c in feature_cols if c.split('_')[0:2]==['bid', 'spread']],
    'ask_spread': [c for c in feature_cols if c.split('_')[0:2]==['ask', 'spread']],
    'total_volume': [c for c in feature_cols if c.split('_')[0:2]==['total', 'volume']],
    'volume_imbalance': [c for c in feature_cols if c.split('_')[0:2]==['volume', 'imbalance']],
    'trade_log_return': [c for c in feature_cols if c.split('_')[0:3]==['trade', 'log', 'return'] and c.split('_')[-1] not in ['time', 'stock']],
    'trade_seconds_in_bucket': [c for c in feature_cols if c.split('_')[0:4]==['trade', 'seconds', 'in', 'bucket']],    
    'trade_size': [c for c in feature_cols if c.split('_')[0:2]==['trade', 'size']],
    'trade_order_count': [c for c in feature_cols if c.split('_')[0:3]==['trade', 'order', 'count']],
    'timeagg': [c for c in feature_cols if c.split('_')[-1]=='time' and 'lambda' not in c],
    'stockagg': [c for c in feature_cols if c.split('_')[-1]=='stock' and 'lambda' not in c],
    'stock_id': ['stock_id'],
    'Bnum': ['Bnum'],
    'Snum': ['Snum'],
    'supBnum': ['supBnum'],
    'supSnum': ['supSnum'],
    'midBSnum': ['midBSnum'],
    'waprollB': [c for c in feature_cols if 'waproll' in c and '_B_' in c],
    'waprollsupB': [c for c in feature_cols if 'waproll' in c and '_supB_' in c],
    'waprollS': [c for c in feature_cols if 'waproll' in c and '_S_' in c],
    'waprollsupS': [c for c in feature_cols if 'waproll' in c and '_supS_' in c],
    'waprollmidBS': [c for c in feature_cols if 'waproll' in c and '_midBS_' in c],
    'waprollup': [c for c in feature_cols if 'waproll' in c and '_up' in c],
    'waprolldown': [c for c in feature_cols if 'waproll' in c and '_down' in c],
    'upnum': ['up50num', 'up25num', 'up75num'],
    'downnum': ['down50num', 'down25num', 'down75num'],
    'candleB': [c for c in feature_cols if 'candle' in c and '_B' in c],
    'candleS': [c for c in feature_cols if 'candle' in c and '_S' in c],
    'candlesupB': [c for c in feature_cols if 'candle' in c and '_supB' in c],
    'candlesupS': [c for c in feature_cols if 'candle' in c and '_supS' in c],
    'candlemidBS': [c for c in feature_cols if 'candle' in c and '_midBS' in c],
    'candleup': [c for c in feature_cols if 'candle' in c and '_up' in c],
    'candledown': [c for c in feature_cols if 'candle' in c and '_down' in c],
    'book_lambda': [c for c in feature_cols if 'book' in c and '_lambda_' in c],
    'price': [c for c in feature_cols if 'price' in c and '_' not in c],
    'size': [c for c in feature_cols if 'size' in c and '_' not in c],
    'order_count': [c for c in feature_cols if 'order_count' in c and 'trade' not in c],
    'seconds_in_bucket': [c for c in feature_cols if 'seconds_in_bucket' in c and 'trade' not in c]
#     'cadlef': [c for c in feature_cols if 'wapcandlef' in c]
#     '150': [c for c in feature_cols if '150' in c],
#     '300': [c for c in feature_cols if '300' in c],
#     '450': [c for c in feature_cols if '450' in c],
}
grouped_features = list(itertools.chain.from_iterable(feature_groups.values()))
len(grouped_features), len(set(grouped_features))

# features: 674


(674, 674)

In [54]:
# check overlap
dummy = []
for k, v in feature_groups.items():
    for c in v:
        if c in dummy:
            print(c)
        else:
            dummy.append(c)

In [55]:
[c for c in feature_cols if c not in grouped_features]

[]

In [56]:
recur_lofo = OptiverRecursiveLOFO(train, feature_cols, group_dict=feature_groups)
recur_lofo.recursive_select()

step 0
#feature 674


/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


base cv mean: 0.22464746018664075


  0%|          | 0/43 [00:00<?, ?it/s]

/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters 

...

/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


base cv mean: 0.22376385771139998


  0%|          | 0/36 [00:00<?, ?it/s]

/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
/home/yuchen/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters 

done!
